In [2]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil import parser
import random
import utilities

%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
from scipy import stats

In [4]:
metas = utilities.loadmetadata()

In [5]:
r = next(metas.iterrows())[1]

In [7]:
events, tweets = utilities.fullgame(r)

utilities.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dt2['time'] = tweets['time'].map(lambda t : parser.parse(t).replace(tzinfo=None))
utilities.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['sentiment'] = df['text'].map(lambda x : classifier.classify(x))
utilities.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [7]:
tweets.index = tweets['time']
events.index = events['WallClockTime']

In [8]:
amonth, aday = tweets['time'].iloc[0].month, tweets['time'].iloc[0].day
startgame = list(events[events['Seconds'] == 0]['WallClockTime'])[0]
endgame = list(events[events['Seconds'] == 0]['WallClockTime'])[-1]

lenhalf = 20*60
lengame = (endgame - startgame).seconds
halfbegin = startgame + timedelta(seconds = lengame/2 - lenhalf/2)
halfend = startgame + timedelta(seconds = lengame/2 + lenhalf/2)

In [9]:
events.index = events['WallClockTime']

In [10]:
filt_tweets = tweets[(tweets['time'] <= halfbegin) | (tweets['time'] >= halfend)]

In [11]:
filt_tweets.shape

(11089, 11)

In [12]:
tweetavgs = filt_tweets['sentnum'].groupby([filt_tweets.index.hour, filt_tweets.index.minute]).mean()
eventavgs = events['Margin'].groupby([events.index.hour, events.index.minute]).mean()

In [13]:
joint = tweetavgs.to_frame().join(eventavgs.to_frame()).fillna(0)

In [16]:
stats.linregress(joint['sentnum'].ix[:-1], joint['Margin'].shift().dropna())

LinregressResult(slope=4.0668512859494523, intercept=5.4195888279151179, rvalue=0.14290413705667965, pvalue=0.14986155256096279, stderr=2.8026728397016218)

In [24]:
def marginsentimentregression(row, sumflag = False):
    # get appropriate data
    events, tweets = utilities.fullgame(r)
    
    # convert indices to time series
    tweets.index = tweets['time']
    events.index = events['WallClockTime']
    
    # calculate the wallclock start and end time of the game
    amonth, aday = tweets['time'].iloc[0].month, tweets['time'].iloc[0].day
    startgame = list(events[events['Seconds'] == 0]['WallClockTime'])[0]
    endgame = list(events[events['Seconds'] == 0]['WallClockTime'])[-1]
    
    # estimate the beginning and end of halftime on the clock
    lenhalf = 20*60
    lengame = (endgame - startgame).seconds
    halfbegin = startgame + timedelta(seconds = lengame/2 - lenhalf/2)
    halfend = startgame + timedelta(seconds = lengame/2 + lenhalf/2)
    
    # filter out half-time tweets to avoid obscuring the regression
    filt_tweets = tweets[(tweets['time'] <= halfbegin) | (tweets['time'] >= halfend)]
    
    # flatten to minute by minute averages
    if sumflag:
        def weightneg(x):
            if x == -1:
                return x*2
            else:
                return x
        filt_tweets['sentnum'] = filt_tweets['sentnum'].map(weightneg)
        tweetavgs = filt_tweets['sentnum'].groupby([filt_tweets.index.hour, filt_tweets.index.minute]).sum()
    else:
        tweetavgs = filt_tweets['sentnum'].groupby([filt_tweets.index.hour, filt_tweets.index.minute]).mean()
    eventavgs = events['Margin'].groupby([events.index.hour, events.index.minute]).mean()
    
    # pair the data
    joint = tweetavgs.to_frame().join(eventavgs.to_frame()).fillna(0)
    
    # compute the regression
    return stats.linregress(joint['sentnum'].ix[:-1], joint['Margin'].shift().dropna())
    